In [1]:
! pip install gliner pytesseract nltk spacy xmltodict textract

  Using cached gliner-0.2.8-py3-none-any.whl (42 kB)
  Using cached onnxruntime-1.18.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.8 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014

In [2]:
from gliner import GLiNER
import spacy
import pytesseract, re, json, nltk, itertools, spacy, difflib, math
model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Harilala Rasoanaivo => person
Rasoanaivo Enterprises => company
Lot II M 92 Antohomadinika => full address
+261 32 22 345 67 => phone number
harilala.rasoanaivo@telma.mg => email
501-02-1234 => Social Security Number


#Simplest Function

In [23]:
def redact_pii(text):
    # Load SpaCy’s NLP model for named entity recognition
    nlp = spacy.load('en_core_web_lg')

    # Process the input text
    doc = nlp(text)

    # Define PII categories (customize as needed)
    pii_categories = [ 'SSN', 'PHONE_NUMBER']

    # Iterate through entities and redact PII
    redacted_text = text


    pii_entities_spacy = [ent.text for ent in doc.ents if ent.label_ in pii_categories]
    labels = ["work", "booking number", "personally identifiable information", "social security number", "driver licence", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number"]
    entities= model.predict_entities(text, labels)
    pii_entities_gliner = [entity["text"] for entity in entities]

    pii_entities = list(set(pii_entities_spacy + pii_entities_gliner))
    for ent in pii_entities:
        redacted_text = redacted_text.replace(ent, '[XXXXXXXXXXX]')

    return redacted_text



document = """

On January 15, 2023, John Doe was visited by a person at 123 Main Street. John's SSN is 1234-56-7. His email is john@gmail.com.
He can be contaced at +91-9988003987. The message left for John was:

John, please get that article on www.linkedin.com to me by 5:00PM
on Jan 9th 2012. 4:00 would be ideal, actually. If you have any
questions, You can reach me at (519)-236-2723x341 or get in touch with
my associate at harold.smith@gmail.com

"""

redacted_document = redact_pii(document)

print(redacted_document)






On January 15, 2023, John Doe was visited by a person at [XXXXXXXXXXX]. John's SSN is [XXXXXXXXXXX]. His email is [XXXXXXXXXXX].
He can be contaced at [XXXXXXXXXXX]. The message left for John was:

John, please get that article on www.linkedin.com to me by 5:00PM 
on Jan 9th 2012. 4:00 would be ideal, actually. If you have any 
questions, You can reach me at [XXXXXXXXXXX] or get in touch with
my associate at [XXXXXXXXXXX]




# Adding Custom Regex

In [31]:
import re

def string_tokenizer(text):
    final_word_list = []
    words_list = text.replace(" ", "\n").split("\n")

    for element in words_list:
        if len(element) >= 2:
            final_word_list.append(element)

    return final_word_list

def similarity(a, b): return difflib.SequenceMatcher(None, a, b).ratio() * 100

def get_regexes():
    rules = {
        "PAN Number": {
            "regex": r"[A-Z]{5}[0-9]{4}[A-Z]{1}"
        }
    }
    return rules

def compile_patterns(patterns):
    if isinstance(patterns, list):
        return [re.compile(pattern) for pattern in patterns]
    else:
        return [re.compile(patterns)]

def custom_pii(text, rules):
    results = []
    all_regexes = {key: re.compile(rule["regex"]) for key, rule in rules.items() if rule["regex"]}

    for key, pattern in all_regexes.items():
        matches = pattern.findall(text)
        if matches:
            results.append({'identifier_class': key, 'result': list(set(matches))})

    return results

def search_pii(text, rules):
    identifiers = custom_pii(text, rules)
    result = {
        "custom PII": identifiers
    }
    return result


{'custom PII': [{'identifier_class': 'PAN Number', 'result': ['AMDPH5087K']}]}


##Way 1

In [42]:
import spacy

def redact_pii(text):
    # Load SpaCy’s NLP model for named entity recognition
    nlp = spacy.load('en_core_web_lg')

    # Process the input text
    doc = nlp(text)

    # Define PII categories and their labels
    pii_labels = {
        'SSN': 'social security number',
        'PHONE_NUMBER': 'phone number',
        'PAN Number': 'PAN number'
    }

    # Initialize the redacted text
    redacted_text = text

    # Extract entities from SpaCy
    pii_entities_spacy = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in pii_labels.keys()]

    # Placeholder for the model's entity prediction
    labels = ["work", "booking number", "personally identifiable information", "social security number", "driver licence", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number"]
    entities = model.predict_entities(text, labels)
    pii_entities_gliner = [(entity["text"], entity["label"]) for entity in entities]

    #Custom regex entities
    regex_result = search_pii(document, get_regexes())
    pii_entities_regex = []
    # Extract and map regex PII entities
    for item in regex_result['custom PII']:

        identifier_class = item['identifier_class']
        if identifier_class in pii_labels:
            for result in item['result']:
                pii_entities_regex.append((result, pii_labels[identifier_class]))

    # Combine entities from both SpaCy and the model
    pii_entities = set(pii_entities_spacy + pii_entities_gliner + pii_entities_regex)

    # Replace each entity with its label in the format <{label}>
    for ent_text, ent_label in pii_entities:
        if ent_text in redacted_text:
            # Find the appropriate label for replacement
            label = pii_labels.get(ent_label, ent_label)  # Use the label from pii_labels dictionary or default to the entity's label
            redacted_text = redacted_text.replace(ent_text, f'<{label}>')

    return redacted_text

# Example document
document = """
On January 15, 2023, John Doe was visited by a person at 123 Main Street. John's SSN is 1234-56-7. His email is john@gmail.com. His PAN number is AMDPH5087K.
He can be contaced at +91-9988003987. The message left for John was:
John, please get that article on www.linkedin.com to me by 5:00PM
on Jan 9th 2012. 4:00 would be ideal, actually. If you have any
questions, You can reach me at (519)-236-2723 or get in touch with
my associate at harold.smith@gmail.com
"""

# Redact PII in the document
redacted_document = redact_pii(document)
print(redacted_document)



On January 15, 2023, John Doe was visited by a person at <full address>. John's SSN is <Social Security Number>. His email is <email>. His PAN number is <PAN number>.
He can be contaced at <phone number>. The message left for John was:
John, please get that article on www.linkedin.com to me by 5:00PM 
on Jan 9th 2012. 4:00 would be ideal, actually. If you have any 
questions, You can reach me at <phone number> or get in touch with
my associate at <email>



##Way 2

In [44]:
import spacy
import re

def redact_pii(text):
    # Load SpaCy’s NLP model for named entity recognition
    nlp = spacy.load('en_core_web_lg')

    # Process the input text
    doc = nlp(text)

    # Initialize the redacted text
    redacted_text = text

    # Extract entities from SpaCy
    pii_entities_spacy = [ent.text for ent in doc.ents if ent.label_ in ['SSN', 'PHONE_NUMBER', 'PAN Number']]

    # Placeholder for the model's entity prediction
    labels = ["work", "booking number", "personally identifiable information", "social security number", "driver licence", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number"]
    entities = model.predict_entities(text, labels)
    pii_entities_gliner = [entity["text"] for entity in entities]

    # Custom regex entities
    regex_result = search_pii(text, get_regexes())
    pii_entities_regex = [result for item in regex_result['custom PII'] for result in item['result']]

    # Combine entities from both SpaCy, model, and regex
    pii_entities = set(pii_entities_spacy + pii_entities_gliner + pii_entities_regex)

    # Replace each entity with [XXXXX] of the same length
    for ent_text in pii_entities:
        redacted_text = re.sub(re.escape(ent_text), '[' + 'X' * len(ent_text) + ']', redacted_text)

    return redacted_text

# Example document
document = """
On January 15, 2023, John Doe was visited by a person at 123 Main Street. John's SSN is 1234-56-7. His email is john@gmail.com. His PAN number is AMDPH5087K.
He can be contaced at +91-9988003987. The message left for John was:
John, please get that article on www.linkedin.com to me by 5:00PM
on Jan 9th 2012. 4:00 would be ideal, actually. If you have any
questions, You can reach me at (519)-236-2723 or get in touch with
my associate at harold.smith@gmail.com
"""

# Redact PII in the document
redacted_document = redact_pii(document)
print(redacted_document)



On January 15, 2023, John Doe was visited by a person at [XXXXXXXXXXXXXXX]. John's SSN is [XXXXXXXXX]. His email is [XXXXXXXXXXXXXX]. His PAN number is [XXXXXXXXXX].
He can be contaced at [XXXXXXXXXXXXXX]. The message left for John was:
John, please get that article on www.linkedin.com to me by 5:00PM 
on Jan 9th 2012. 4:00 would be ideal, actually. If you have any 
questions, You can reach me at [XXXXXXXXXXXXXX] or get in touch with
my associate at [XXXXXXXXXXXXXXXXXXXXXX]

